In [3]:
# this code compiles all Textgrids from smaller files to a big one 'all.TextGrid', which will be used further

from praatio import tgio


filenames = ['game_885-1785_UPD.TextGrid', 'game_1785-2605_UPD.TextGrid', 'game_2605-3500_UPD.TextGrid', 'game_3500-4400_UPD.TextGrid', 'game_4400-5300_UPD.TextGrid']
a = tgio.openTextgrid('game_1-885_UPD.TextGrid')
for file in filenames:
    b = tgio.openTextgrid(file)
    combinedTG = a.appendTextgrid(b, True)
combinedTG.save("all.TextGrid")

ValueError: min() arg is an empty sequence

In [ ]:
# this code creates a database 'oredered.db' for ordered utterances, otherwise they will not be saved
import sqlite3

conn = sqlite3.connect('oredered.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS new(start_point float, replic text, speaker integer, language text, inter integer)')
conn.close()

In [ ]:
# this code creates a database 'oredered.db', consisted from speakers' utterances ordered by time
# structure of the database: start point, utterance, speaker, language, instance of intersententional code-switch
# languages:
#     'R' - russian
#     'Y' - yakut
#     'CR' - game names
#     'CS' - utterances with intrasententional code-switch
#     'N' - not clear utterances

def utterances_db():
    conn = sqlite3.connect('utterances.db')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS utterances(start_point float, utterance text, speaker integer, language text, CS integer)')
#   list for time points of code-switch
    cs_list = []
    tg = tgio.openTextgrid('all.TextGrid')
#   reading tier with intersententional code-sitches
    cs = tg.tierDict[tg.tierNameList[20]]
    cs_list.extend([entry[0] for entry in cs.entryList])
#   iterating through speakers
    for i in range (0, 16, 5):
#       list of speakers' starting points
        starting_point = []
#       list of utterances
        utterance = []
        a = tg.tierDict[tg.tierNameList[i]]
#       reading tier with words in russian
        rus = tg.tierDict[tg.tierNameList[i + 1]]
#       reading tier with words in yakut
        ykt = tg.tierDict[tg.tierNameList[i + 2]]
        rus_start_point = [entry[0] for entry in rus.entryList]
        rus_end_point = [entry[1] for entry in rus.entryList]
        ykt_start_point = [entry[0] for entry in ykt.entryList]
        ykt_end_point = [entry[1] for entry in ykt.entryList]
#       distinguishing languages and marking code-switch
        for start_point, end_point, label in a.entryList:
            inter = 0
            if s_point in cster_list:
                inter = 1
            if start_point in rus_start_points and end_point in rus_end_point:
                language = 'R'
            elif start_point in ykt_start_point and end_point in ykt_end_point:
                language = 'Y'
            elif start_point in rus_start_point and end_point in ykt_end_point:
                language = 'CS'
            elif start_point in ykt_start_point and end_point in rus_end_point:
                language = 'CS'
            else:
                language = 'N'
            c.execute('INSERT INTO utterances VALUES (?, ?, ?, ?, ?)', (start_point, label, i, language, inter))  
        conn.commit()
#   saving ordered database
    c.execute('ATTACH DATABASE "oredered.db" as oredered')
    c.execute('INSERT INTO oredered SELECT * FROM utterances ORDER BY start_point ASC')
    conn.commit()
    conn.close()
    
utterances()


In [ ]:
# this code creates a databes of utterance pairs 'turns.db': an utterance followed by other speaker's utterance
# 'L[eft]' is for everything connected with a person who talked earlier, 'R[ight]' - later
# structure: start point L, intersententional code-switch L, speaker L, utterance L, language L, and the same for the spaker R, but reversed
def turns():
    conn = sqlite3.connect('ordered.db')
    c = conn.cursor()   
    c.execute('SELECT * FROM ordered')
    a = c.fetchall()
    conn.close()
    n = len(a)
    conn = sqlite3.connect('turns.db')
    c = conn.cursor() 
    c.execute('CREATE TABLE IF NOT EXISTS turns(start_pointL float, CSL integer, speakerL integer, replicL text, languageL text, languageR text, replicR text, speakerR integer, CSR integer, start_pointR float)')
    for i in range (0, n - 1):
        if a[i][2] != a[i + 1][2]:
            c.execute('INSERT INTO turns VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (a[i][0], a[i][4], a[i][2], a[i][1], a[i][3], a[i + 1][3], a[i + 1][1], a[i + 1][2], a[i + 1][4], a[i + 1][0]))
        conn.commit()
    conn.close()

turns()